In [2]:
import pandas as pd
import pickle
import numpy as np
from sklearn.utils import class_weight
from sklearn.preprocessing import OneHotEncoder
import random
import datetime

In [3]:
w_length = 300

In [1]:
def splitSeconds(n, country, t, seconds, samplerate):
    length = seconds * samplerate
    data = pickle.load( open( "Raw Track Data\\" + country + "_" + t + ".p", "rb" ) )
    tracks = data.track_id.unique()
    tracks = np.random.choice(tracks, size=n, replace=False)
    trackFeats = data[data.track_id.isin(tracks)]
    del data
    dur = trackFeats.iloc[:,1]
    long = trackFeats.loc[trackFeats.index.repeat(dur * samplerate)].reset_index(drop = True)
    del trackFeats
    long = long.sort_values(by = ["track_id", "start"])
    long['change'] = long.track_id.eq(long.track_id.shift())
    change = long[long.change == False].index
    long = long.iloc[:, 5:30]
    indices = np.concatenate((np.arange(0, long.shape[0], length), change))
    indices = np.sort(indices)
    indices = np.unique(indices)
    partition = np.split(np.array(long), indices)
    del long
    samples = []
    for i in partition:
        if i.shape[0] == length:
            samples = samples + [i]
    samples = np.stack(samples)
    return samples, np.repeat(np.array([country]), samples.shape[0])

In [4]:
def getSamples(train_n, val_n, seconds, samplerate, countriesOfInterest,
               enc, verbose = 0):
    train_labels = pd.DataFrame()
    val_labels = pd.DataFrame()
    train_x = []
    train_labels = []
    val_x = []
    val_labels = []
    for country in countriesOfInterest:
        if verbose > 0:
            print("getting",country)
        x1, y1 = splitSeconds(train_n, country, "train", seconds, samplerate)
        x2, y2 = splitSeconds(val_n, country, "val", seconds, samplerate)
        train_x = train_x + x1.tolist()
        train_labels = train_labels + y1.tolist()
        del x1, y1
        val_x = val_x + x2.tolist()
        val_labels = val_labels + y2.tolist()
        del x2, y2
    #train_x = np.array(train_x)
    y = np.dstack(train_x)
    del train_x
    train_x = np.rollaxis(y,-1)
    del y
    train_labels = np.array(train_labels)
    #val_x = np.array(val_x)
    y = np.dstack(val_x)
    del val_x
    val_x = np.rollaxis(y,-1)
    del y
    val_labels = np.array(val_labels)
    class_weights = class_weight.compute_class_weight('balanced',
                                                     countriesOfInterest,
                                                     list(train_labels))
    train_labels = enc.transform(np.array(train_labels).reshape(-1,1)).toarray()
    val_labels = enc.transform(np.array(val_labels).reshape(-1,1)).toarray()
    return train_x, train_labels, val_x, val_labels, class_weights

In [11]:
countriesOfInterest = ["ZA", "EG", "TW", "JP", "DK", "FI", "US", "CA", "AU", "NZ", "BR", "CO"]
train_n = 10
seconds = 10
samplerate = 10 
train_labels = pd.DataFrame()
val_labels = pd.DataFrame()
train_x = []
train_labels = []
val_x = []
val_labels = []
for country in countriesOfInterest:
    x1, y1 = splitSeconds(train_n, country, "train", seconds, samplerate)
    train_x = train_x + x1.tolist()
    train_labels = train_labels + y1.tolist()

In [21]:
x1.shape

(201, 100, 25)

In [23]:
np.array(train_x)

array([[[-6.0000e+01,  1.0000e+00,  2.2400e-01, ...,  9.7300e-01,
         -1.0640e+01, -7.2280e+00],
        [-6.0000e+01,  1.0000e+00,  2.2400e-01, ...,  9.7300e-01,
         -1.0640e+01, -7.2280e+00],
        [-3.4087e+01,  2.4400e-01,  1.0000e+00, ...,  1.6004e+01,
          1.1418e+01, -6.5190e+00],
        ...,
        [-7.6450e+00,  3.2000e-02,  1.1000e-02, ..., -5.2380e+00,
         -4.5440e+01,  4.2320e+00],
        [-7.6450e+00,  3.2000e-02,  1.1000e-02, ..., -5.2380e+00,
         -4.5440e+01,  4.2320e+00],
        [-2.4297e+01,  2.6300e-01,  6.8600e-01, ..., -1.4858e+01,
         -8.1790e+00,  1.0888e+01]],

       [[-2.5327e+01,  1.0000e+00,  4.6900e-01, ...,  3.9760e+00,
         -1.1933e+01,  1.7507e+01],
        [-2.5327e+01,  1.0000e+00,  4.6900e-01, ...,  3.9760e+00,
         -1.1933e+01,  1.7507e+01],
        [-2.6054e+01,  1.0000e+00,  8.2300e-01, ..., -8.8070e+00,
         -1.0291e+01,  1.1350e+01],
        ...,
        [-1.9238e+01,  1.0000e+00,  6.1500e-01, ..., -

In [8]:
sample[0].shape

(163, 100, 25)

In [87]:
country = "ZA"
t = "train"
n = 50
print("getting file")
data = pickle.load( open( "Raw Track Data\\" + country + "_" + t + ".p", "rb" ) )
tracks = data.track_id.unique()
tracks = np.random.choice(tracks, size=n, replace=False)
samples = []

getting file


In [91]:
def splitSeconds(n, country, t, seconds, samplerate):
    length = seconds * samplerate
    data = pickle.load( open( "Raw Track Data\\" + country + "_" + t + ".p", "rb" ) )
    tracks = data.track_id.unique()
    tracks = np.random.choice(tracks, size=n, replace=False)
    trackFeats = data[data.track_id.isin(tracks)]
    dur = trackFeats.iloc[:,1]
    long = trackFeats.loc[trackFeats.index.repeat(dur * samplerate)].reset_index(drop = True)
    long['change'] = long.track_id.eq(long.track_id.shift())
    change = long[long.change == False].index
    long = long.iloc[:, 5:30]
    indices = np.concatenate((np.arange(0, long.shape[0], length), change))
    indices = np.sort(indices)
    indices = np.unique(indices)
    partition = np.split(np.array(long), indices)
    samples = []
    for i in partition:
        if i.shape[0] == length:
            samples = samples + [i]
    samples = np.stack(samples)
    return samples, np.repeat(np.array([country]), samples.shape[0])

In [90]:
get

array(['ZA', 'ZA', 'ZA', ..., 'ZA', 'ZA', 'ZA'], dtype='<U2')

In [98]:
countriesOfInterest = ["ZA", "EG", "TW", "JP", "DK", "FI", "US", "CA", "AU", "NZ", "BR", "CO"]

train_x, train_labels, val_x, val_labels, class_weights = getSamples(200, 50)

ZA
getting file
repeating
partitioning
checking
getting file
repeating
partitioning
checking
collected
EG
getting file
repeating


KeyboardInterrupt: 

In [96]:
class_weights

array([1.11811061, 1.11369618, 0.96462303, 1.06331897, 0.85851449,
       0.95114804, 0.8683304 , 1.22721152, 1.29650908, 0.73816199,
       1.1909429 , 0.92399782])